In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Mela Sombillo csombillo1@tonikbank.com\JIRA ticket and Ticket Summary\JIRA ticket and Ticket Summary.csv")

In [3]:
df.head()

,customerId,loanAccountNumber,disbursedLoanAmount,dateOfBirth,Age,name,Ref1_contact,Ref2_contact,tinNumber,docType,...,custLoanJourneyZipCode,custLoanJourneyCity,custLoanJourneyProvince,geolocation,loan_app_d,Loan app Time,employmentstatus,natureofwork,industryDescription,subIndustryDescription
0,1152544,NaN,NaN,20-Nov-66,58,DAVID ROPETA LABUSON JR,639063152204,639487565079,353766854,Philippines - Social Security Card #1,...,4600,DAET,CAMARINES NORTE,1.280894 | 103.849252,29-11-2021,T10:22:59,UNEMPLOYED,NaN,Services,Others
1,1537578,NaN,NaN,15-Oct-89,35,JUDITH PUNZAL SANTIAGO,639502218777,639350583785,353766854,Philippines - UMID Card (2011),...,3102,SAN LEONARDO,NUEVA ECIJA,15.3681911 | 120.9647815,10-08-2022,T12:24:03,OVERSEAS FILIPINO WORKER,Other Non Professional Services,Manufacturing,Others
2,1901410,NaN,NaN,21-Mar-97,27,MARA LADY LIM LAYSON,639368851229,639852010639,353766854,Philippines - Id Card (2020),...,9805,ISULAN,SULTAN KUDARAT,6.6328044 | 124.5983674,15-02-2023,T11:39:45,BUSINESS OWNER/SOLE PROPRIETOR,Owner,Services,Grooming (Salon / Parlour)
3,1812444,NaN,NaN,23-Dec-99,25,GIRREL PALOR AUMAN,639354963561,639700932592,223724698,Philippines - Health Card,...,8000,DAVAO CITY,DAVAO DEL SUR,7.0532679 | 125.6040032,11-12-2022,T09:03:58,SALARIED,Call Center Agent/Tele Marketer,Services,Call Centre / BPO
4,1271268,NaN,NaN,01-Aug-98,26,CHRISTINE MARY GULANE OTRERA,639922765359,639185878605,205366921,Philippines - UMID Card #1,...,6045,TALISAY,CEBU,0.00 | 0.00,12-03-2022,T05:25:44,SELF EMPLOYED,Other Professional services,Services,Call Centre / BPO


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097 entries, 0 to 1096
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   customerId               1097 non-null   int64  
 1   loanAccountNumber        2 non-null      float64
 2   disbursedLoanAmount      2 non-null      float64
 3   dateOfBirth              1097 non-null   object 
 4   Age                      1097 non-null   int64  
 5   name                     1027 non-null   object 
 6   Ref1_contact             1097 non-null   int64  
 7   Ref2_contact             1097 non-null   int64  
 8   tinNumber                1097 non-null   int64  
 9   docType                  1094 non-null   object 
 10  ID Type                  1097 non-null   object 
 11  docNumber                1093 non-null   object 
 12  email                    1097 non-null   object 
 13  tsa_zip                  1097 non-null   int64  
 14  tsa_city                

In [5]:
df['customerId'] = df['customerId'].astype(str)
a = tuple(df['customerId'])
a[0:5]

('1152544', '1537578', '1901410', '1812444', '1271268')

In [7]:
df1 = client.query(f"""select * from prj-prod-dataplatform.dl_dynamo_db_raw.tdbk_jiraticket_trc where custId in {a}""").to_dataframe()
df1.shape                
                   

(0, 15)

In [12]:
sq = f"""with b1 as 
(select digitalLoanAccountId, cast(customerId as string) customerId , applicationStatus, Stage0_Status, stage1Status, stage0FailureReason, stage1FailureReason
from`risk_credit_mis.loan_master_table` 
where cast(customerId as string) 
in  {a}
)
select * from b1
---select * from prj-prod-dataplatform.dl_loans_db_raw.tdbk_loan_jira_details b2 
---# inner join b1 on b1.digitalLoanAccountId = b2.digitalLoanAccountId
"""

df1 = client.query(sq).to_dataframe()
df1

,digitalLoanAccountId,customerId,applicationStatus,Stage0_Status,stage1Status,stage0FailureReason,stage1FailureReason
0,3b65d92c-072d-494b-86ab-4d600acc6743,1920742,RESET,None,None,None,None
1,5153198a-ab0d-4314-8a2e-56263b1796a6,1904481,RESET,None,None,None,None
2,6a07906c-6bef-4df8-9135-94ea380d05f0,1791544,RESET,None,None,None,None
3,26c6912a-c986-4c62-a1ad-89c282e355bf,2064863,RESET,None,None,None,None
4,754b582c-aa82-488a-b342-61ae49e52793,2209956,CANCELLED,None,None,None,None
...,...,...,...,...,...,...,...
1497,ce603109-89c6-46d1-8262-6fd2730e055f,2332346,REJECT,None,None,None,None
1498,8216b71e-3c3a-44e5-9221-c06107ee7f73,2336410,REJECT,None,None,None,None
1499,b0cbc02c-16f5-4ba0-8229-4862578e2449,2428912,REJECT,None,None,None,None
1500,42b9a692-5343-4b4d-92f2-c98548cc6020,1430054,EXPIRED,REJECT,None,Applicant age is less then minimum age|Applica...,None


In [24]:
sq = f"""WITH
W_TICKETS_BASE1
AS
(
  select 
  b.issue_key,
  b.summary,
  REGEXP_EXTRACT(b.summary, r'\d+') AS customerid,
    CASE 
    WHEN ARRAY_LENGTH(SPLIT(summary, '|')) >= 2 THEN TRIM(SPLIT(summary, '|')[OFFSET(1)])
    ELSE NULL  -- or any default value you want to use when the format is unexpected
  END AS customer_id,
  REPLACE( REPLACE( REPLACE( REGEXP_REPLACE(b.summary, '[0-9]', '') , '|', ''), '  ', ' '), '  ', ' ') as Type, 
  b.created_date as Ticket_Creation_Dttm
  FROM
  prj-prod-dataplatform.jira_raw.cc_tickets b
  WHERE
  b.created_date < CURRENT_DATE AND EXTRACT(DATE FROM B.created_date) >= DATE '2023-08-01'
),
b1 as 
(select * from W_TICKETS_BASE1 where customerid is not null and length(customerid) = 7 )
select * from b1 where customerid in {a}
---
"""
print(sq)
df1 = client.query(sq).to_dataframe()

WITH
W_TICKETS_BASE1
AS
(
  select 
  b.issue_key,
  b.summary,
  REGEXP_EXTRACT(b.summary, r'\d+') AS customerid,
    CASE 
    WHEN ARRAY_LENGTH(SPLIT(summary, '|')) >= 2 THEN TRIM(SPLIT(summary, '|')[OFFSET(1)])
    ELSE NULL  -- or any default value you want to use when the format is unexpected
  END AS customer_id,
  REPLACE( REPLACE( REPLACE( REGEXP_REPLACE(b.summary, '[0-9]', '') , '|', ''), '  ', ' '), '  ', ' ') as Type, 
  b.created_date as Ticket_Creation_Dttm
  FROM
  prj-prod-dataplatform.jira_raw.cc_tickets b
  WHERE
  b.created_date < CURRENT_DATE AND EXTRACT(DATE FROM B.created_date) >= DATE '2023-08-01'
),
b1 as 
(select * from W_TICKETS_BASE1 where customerid is not null and length(customerid) = 7 )
select * from b1 where customerid in ('1152544', '1537578', '1901410', '1812444', '1271268', '1433044', '2073203', '2078567', '2171735', '2261673', '2384172', '1813301', '1941310', '2358052', '2424919', '1450849', '1256969', '1729826', '1689864', '2387056', '1928792', '186

In [25]:
df = df.merge(df1, left_on='customerId', right_on='customerid', how = 'left')
df

,customerId,loanAccountNumber,disbursedLoanAmount,dateOfBirth,Age,name,Ref1_contact,Ref2_contact,tinNumber,docType,...,employmentstatus,natureofwork,industryDescription,subIndustryDescription,issue_key,summary,customerid,customer_id,Type,Ticket_Creation_Dttm
0,1152544,NaN,NaN,20-Nov-66,58,DAVID ROPETA LABUSON JR,639063152204,639487565079,353766854,Philippines - Social Security Card #1,...,UNEMPLOYED,NaN,Services,Others,NaN,NaN,NaN,NaN,NaN,NaT
1,1537578,NaN,NaN,15-Oct-89,35,JUDITH PUNZAL SANTIAGO,639502218777,639350583785,353766854,Philippines - UMID Card (2011),...,OVERSEAS FILIPINO WORKER,Other Non Professional Services,Manufacturing,Others,NaN,NaN,NaN,NaN,NaN,NaT
2,1901410,NaN,NaN,21-Mar-97,27,MARA LADY LIM LAYSON,639368851229,639852010639,353766854,Philippines - Id Card (2020),...,BUSINESS OWNER/SOLE PROPRIETOR,Owner,Services,Grooming (Salon / Parlour),NaN,NaN,NaN,NaN,NaN,NaT
3,1812444,NaN,NaN,23-Dec-99,25,GIRREL PALOR AUMAN,639354963561,639700932592,223724698,Philippines - Health Card,...,SALARIED,Call Center Agent/Tele Marketer,Services,Call Centre / BPO,NaN,NaN,NaN,NaN,NaN,NaT
4,1271268,NaN,NaN,01-Aug-98,26,CHRISTINE MARY GULANE OTRERA,639922765359,639185878605,205366921,Philippines - UMID Card #1,...,SELF EMPLOYED,Other Professional services,Services,Call Centre / BPO,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1100,2365853,NaN,NaN,29-Mar-00,24,CLIFFORD JOHN ESPIRITU DAPON,639620662963,639693089519,353766854,Philippines - ePassport (2016),...,SALARIED,Other Non Professional Services,Services,Manpower / Manning Agencies (Construction Indu...,CC-2267005,Onboarding | 2365853 | mandatory CDD Review,2365853,2365853,Onboarding mandatory CDD Review,2024-01-11 20:42:14
1101,2372897,NaN,NaN,30-Nov-01,23,JHON RAFAEL MARAVELLES NEPOMUCENO,639513663161,639109105578,353766854,Philippines - Driving License (2017) #3,...,BUSINESS OWNER/SOLE PROPRIETOR,Owner,Services,Others,CC-2279848,Onboarding | 2372897 | mandatory CDD Review,2372897,2372897,Onboarding mandatory CDD Review,2024-01-18 23:46:21
1102,2413494,NaN,NaN,29-May-00,24,ALVIN GUINABO MOJANA,639198614676,639640115307,353766854,Philippines - Id Card (2020),...,SELF EMPLOYED,Farmer/Fisherman,Agriculture,Agriculture / Horticulture,CC-2355076,Onboarding | 2413494 | Manual ID Upload - CDD ...,2413494,2413494,Onboarding Manual ID Upload - CDD Review,2024-02-27 22:21:46
1103,2475539,NaN,NaN,02-Jun-05,19,JOHN ANTHONY RIZALDO GOLPEO,639516830454,639476159086,353766854,Philippines - Id Card (2020),...,STUDENT,NaN,Trading,Others,CC-2461230,Onboarding | 2475539 | mandatory CDD Review,2475539,2475539,Onboarding mandatory CDD Review,2024-04-21 18:29:23


In [26]:
df.to_excel("jira_ticketandTicketsummary_data_20240513.xlsx", index = False)